In [7]:
import gurobipy
from gurobipy import *
from contextlib import contextmanager
import sys, os
import pandas as pd
import numpy as np
from __future__ import division
from scipy import stats as st
from scipy.special import comb
lp = pd.read_csv('data114.csv', sep = ',')
lp = pd.DataFrame.as_matrix(lp)

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout


#1.14#
categories, minimum, maximum = multidict({
  'machine_hours': [0, lp[1,0]],
  'cash_available':      [0, lp[2,0]] })

products, cost = multidict({
  'x1': -lp[0,1],
  'x2': -lp[0,2] })


constraints = {
  ('x1', 'machine_hours'): lp[1,1],
  ('x1', 'cash_available'):  lp[2,1],
  ('x2',   'machine_hours'): lp[1,2],
  ('x2',   'cash_available'):  lp[2,2], }

m = Model("schedule")


totalCost = m.addVars(categories, lb=minimum, ub=maximum, name="totalCost")


buy = m.addVars(products, name="buy")

m.setObjective(buy.prod(cost), GRB.MINIMIZE)

m.addConstrs(
    (quicksum(constraints[p,c] * buy[p] for p in products) == totalCost[c]
     for c in categories), "_")

def printSolution():
    if m.status == GRB.Status.OPTIMAL:
        print('\nCost: %g' % -m.objVal)
        print('\nBuy:')
        buyx = m.getAttr('x', buy)
        totalCostx = m.getAttr('x', totalCost)
        for p in products:
            if buy[p].x > 0.0001:
                print('%s %g' % (p, buyx[p]))
        print('\nSchedule:')
        for c in categories:
            print('%s %g' % (c, totalCostx[c]))
    else:
        print('No solution')


with suppress_stdout():m.optimize();
#m.optimize()
printSolution()
m.write("problem_1_14.lp")


Cost: 20

Buy:
x1 6.66667

Schedule:
machine_hours 20
cash_available 2


In [8]:
#1.15(a) THIS IS EASY TO FORMULATE INTO A SINGLE PROBLEM#
lp = pd.read_csv('data115a.csv', sep = ',')
lp = pd.DataFrame.as_matrix(lp)


categories, minimum, maximum = multidict({
  'assembly': [0, lp[1,0]],
  'testing':[0, lp[2,0]],})
products, cost = multidict({
  'p1': -lp[0,1],
  'p2': -lp[0,2]})
constraints = {
  ('p1', 'assembly'): lp[1,1],
  ('p1', 'testing'):  lp[2,1],
  ('p2',   'assembly'): lp[1,2],
  ('p2',   'testing'): lp[2,2]}
m = Model("schedule")

totalCost = m.addVars(categories, lb=minimum, ub=maximum, name="totalCost")
buy = m.addVars(products, name="buy")
m.setObjective(buy.prod(cost), GRB.MINIMIZE)
m.addConstrs(
    (quicksum(constraints[p,c] * buy[p] for p in products) == totalCost[c]
     for c in categories), "_")
def printSolution():
    if m.status == GRB.Status.OPTIMAL:
        print('\nCost: %g' % -m.objVal)
        print('\nBuy:')
        buyx = m.getAttr('x', buy)
        totalCostx = m.getAttr('x', totalCost)
        for p in products:
            if buy[p].x > 0.0001:
                print('%s %g' % (p, buyx[p]))
        print('\nSchedule:')
        for c in categories:
            print('%s %g' % (c, totalCostx[c]))
    else:
        print('No solution')
with suppress_stdout():m.optimize();
#m.optimize()
printSolution()
m.write("problem_1_15a.lp")


Cost: 2808

Buy:
p1 360

Schedule:
testing 45
assembly 90


In [9]:
print("1.15(b.i) THIS IS EASY TO FORMULATE INTO A SINGLE PROBLEM")
categories, minimum, maximum = multidict({
  'assembly': [0, lp[1,0]],
  'testing':[0, lp[2,0]],
  'overtime':[0, lp[3,0]] })
products, cost = multidict({
  'p1': -lp[0,1],
  'p2': -lp[0,2],
  'p3': lp[0,3]})
constraints = {
  ('p1', 'assembly'):lp[1,1],
  ('p1', 'testing'):lp[1,2],
  ('p1','overtime'):lp[1,3],
  ('p2',   'assembly'):lp[2,1],
  ('p2',   'testing'):lp[2,2],
  ('p2',   'overtime'):lp[2,3], 
  ('p3',   'assembly'):lp[3,1],
  ('p3',   'testing'): lp[3,2],
  ('p3',   'overtime'):lp[3,3]}
m = Model("schedule")

totalCost = m.addVars(categories, lb=minimum, ub=maximum, name="totalCost")
buy = m.addVars(products, name="buy")
m.setObjective(buy.prod(cost), GRB.MINIMIZE)
m.addConstrs(
    (quicksum(constraints[p,c] * buy[p] for p in products) == totalCost[c]
     for c in categories), "_")
def printSolution():
    if m.status == GRB.Status.OPTIMAL:
        print('\nCost: %g' % -m.objVal)
        print('\nBuy:')
        buyx = m.getAttr('x', buy)
        totalCostx = m.getAttr('x', totalCost)
        for p in products:
            if buy[p].x > 0.0001:
                print('%s %g' % (p, buyx[p]))
        print('\nSchedule:')
        for c in categories:
            print('%s %g' % (c, totalCostx[c]))
    else:
        print('No solution')
with suppress_stdout():m.optimize();
#m.optimize()
printSolution()
m.write("problem_1_15bi.lp")

1.15(b.i) THIS IS EASY TO FORMULATE INTO A SINGLE PROBLEM

Cost: 1872

Buy:
p1 240

Schedule:
overtime 0
testing 80
assembly 60


In [10]:
print( "1.15(b.ii) THIS IS NOT EASY to formulate into a linear programming problem, we must divide it into two cases:")
print("\n"+"Case 1: NO DISCOUNT implies constraint 1.2x + 0.9y <= 300 is used")
lp = pd.read_csv('data115b.csv', sep = ',')
lp = pd.DataFrame.as_matrix(lp)

categories, minimum, maximum = multidict({
  'assembly': [0, 90.00],
  'testing':[0, 380.0],
  'overtime':[0,50]})
products, cost = multidict({
  'p1': -lp[0,1],
  'p2': -lp[0,2],
  'p3': lp[0,3]})
constraints = {
  ('p1', 'assembly'):lp[1,1],
  ('p1', 'testing'):lp[1,2],
  ('p1','overtime'):lp[1,3],
  ('p2',   'assembly'):lp[2,1],
  ('p2',   'testing'):lp[2,2],
  ('p2',   'overtime'):lp[2,3], 
  ('p3',   'assembly'):lp[3,1],
  ('p3',   'testing'): lp[3,2],
  ('p3',   'overtime'): lp[3,3]}
m = Model("schedule")

totalCost = m.addVars(categories, lb=minimum, ub=maximum, name="totalCost")
buy = m.addVars(products, name="buy")
m.setObjective(buy.prod(cost), GRB.MINIMIZE)
m.addConstrs(
    (quicksum(constraints[p,c] * buy[p] for p in products) == totalCost[c]
     for c in categories), "_")
def printSolution():
    if m.status == GRB.Status.OPTIMAL:
        print('\nCost: %g' % -m.objVal)
        print('\nBuy:')
        buyx = m.getAttr('x', buy)
        totalCostx = m.getAttr('x', totalCost)
        for p in products:
            if buy[p].x > 0.0001:
                print('%s %g' % (p, buyx[p]))
        print('\nSchedule:')
        for c in categories:
            print('%s %g' % (c, totalCostx[c]))
    else:
        print('No solution')
with suppress_stdout():m.optimize();
#m.optimize()
printSolution()
m.write("problem_1_15ii(case1).lp")
#########################################################################################################
print("\n"+"Case 2: DISCOUNT implies constraint 1.2x + 0.9 >= 300 (written as -1.2x - 0.9 <= -300) is used"+"\n")
categories, minimum, maximum = multidict({
  'assembly': [0, 90.00],
  'testing':[-GRB.INFINITY, -220],
  'overtime':[0,50]})
products, cost = multidict({
  'p1': (0.9*1.2-9),
  'p2': (0.9**2-8),
  'p3': 7})
constraints = {
  ('p1', 'assembly'):0.25,
  ('p1', 'testing'):(0.125-1.2),
  ('p1','overtime'):0,
  ('p2',   'assembly'):(1.0/3.0),
  ('p2',   'testing'):(1.0/3.0)-(0.9),
  ('p2',   'overtime'):0, 
  ('p3',   'assembly'):-1.0,
  ('p3',   'testing'): 0,
  ('p3',   'overtime'): 1.0}
m = Model("schedule")

totalCost = m.addVars(categories, lb=minimum, ub=maximum, name="totalCost")
buy = m.addVars(products, name="buy")
m.setObjective(buy.prod(cost), GRB.MINIMIZE)
m.addConstrs(
    (quicksum(constraints[p,c] * buy[p] for p in products) == totalCost[c]
     for c in categories), "_")
def printSolution():
    if m.status == GRB.Status.OPTIMAL:
        print('\nCost: %g' % -m.objVal)
        print('\nBuy:')
        buyx = m.getAttr('x', buy)
        totalCostx = m.getAttr('x', totalCost)
        for p in products:
            if buy[p].x > 0.0001:
                print('%s %g' % (p, buyx[p]))
        print('\nSchedule:')
        for c in categories:
            print('%s %g' % (c, totalCostx[c]))
    else:
        print('No solution')

with suppress_stdout():m.optimize();
#m.optimize()
printSolution()
print("Case 1 has a better cost")
m.write("problem_1_15ii(case2).lp")

1.15(b.ii) THIS IS NOT EASY to formulate into a linear programming problem, we must divide it into two cases:

Case 1: NO DISCOUNT implies constraint 1.2x + 0.9y <= 300 is used

Cost: 489.057

Buy:
p2 67.9245

Schedule:
overtime 0
testing 83.7736
assembly 90

Case 2: DISCOUNT implies constraint 1.2x + 0.9 >= 300 (written as -1.2x - 0.9 <= -300) is used


Cost: 4085.2

Buy:
p3 50
p1 560

Schedule:
overtime 50
testing -602
assembly 90
Case 1 has a better cost


In [11]:
##### print("1.16")

print("PROFIT = $38*(4*p1 + p2 + 3*p4)  + $33*(3*p1 + p2 + 4*p3)"+"\n"
      +"COST = -$(51*p1 + 11*p2 + 40*p3)"+"\n"+"Objective function : NET REVENUE = PROFIT - COST" 
      +": 251*p1 + 71*p2 + 256*p3")
lp = pd.read_csv('data116.csv', sep = ',')
lp = pd.DataFrame.as_matrix(lp)


categories, minimum, maximum = multidict({
  'In_A': [0, lp[1,0]],
  'In_B':[0, lp[2,0]] })
products, cost = multidict({
  'process_1': -lp[0,1],
  'process_2': -lp[0,2],
  'process_3': -lp[0,3]})
constraints = {
  ('process_1', 'In_A'):lp[1,1],
  ('process_1', 'In_B'):lp[2,1],
  ('process_2',   'In_A'):lp[1,2],
  ('process_2',   'In_B'):lp[2,2],
  ('process_3',   'In_A'):lp[1,3], 
  ('process_3',   'In_B'):lp[2,3]}
m = Model("schedule")

totalCost = m.addVars(categories, lb=minimum, ub=maximum, name="totalCost")
buy = m.addVars(products, name="buy")
m.setObjective(buy.prod(cost), GRB.MINIMIZE)
m.addConstrs(
    (quicksum(constraints[p,c] * buy[p] for p in products) == totalCost[c]
     for c in categories), "_")
def printSolution():
    if m.status == GRB.Status.OPTIMAL:
        print('\nNet Revenue: %g' % -m.objVal)
        print('\nSchedule:')
        buyx = m.getAttr('x', buy)
        totalCostx = m.getAttr('x', totalCost)
        for p in products:
            if buy[p].x > 0.0001:
                print('%s: %g' % (p, buyx[p]))
        print('\nOil Used:')
        for c in categories:
            print('%s: %g' % (c, totalCostx[c]))
    else:
        print('No solution')
with suppress_stdout():m.optimize();
#m.optimize()
printSolution()
m.write("problem_1_16.lp")

PROFIT = $38*(4*p1 + p2 + 3*p4)  + $33*(3*p1 + p2 + 4*p3)
COST = -$(51*p1 + 11*p2 + 40*p3)
Objective function : NET REVENUE = PROFIT - COST: 251*p1 + 71*p2 + 256*p3

Net Revenue: 3.375e+08

Schedule:
process_3: 1.5e+06
process_2: 500000

Oil Used:
In_A: 8e+06
In_B: 5e+06
